In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

24/03/07 18:23:43 WARN Utils: Your hostname, Jonathans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.30 instead (on interface en0)
24/03/07 18:23:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/07 18:23:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

home_df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'),
                         sep = ',',
                         header= True)
home_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [5]:
# 2. Create a temporary view of the DataFrame.

home_df.createOrReplaceTempView('home_sales')
home_df

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_4_bed = spark.sql('''
    SELECT 
        YEAR(date) AS yr_sold, 
        ROUND(AVG(price), 2) AS average_price 
    FROM 
        home_sales
    WHERE
        bedrooms = 4
    GROUP BY
        YEAR(date)
    ORDER BY
        yr_sold
    ''')

avg_4_bed.show()


+-------+-------------+
|yr_sold|average_price|
+-------+-------------+
|   2019|     300263.7|
|   2020|    298353.78|
|   2021|    301819.44|
|   2022|    296363.88|
+-------+-------------+



In [8]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_home_price = spark.sql('''
    SELECT 
        date_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    WHERE 
        bedrooms = 3 
        AND 
        bathrooms = 3
    GROUP BY 
        date_built
    ORDER BY 
        date_built
''')

avg_home_price.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

avg_home_price = spark.sql('''
    SELECT 
        date_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    WHERE 
        bedrooms = 3 
        AND 
        bathrooms = 3
        AND 
        floors = 2
        AND 
        sqft_living >= 2000
    GROUP BY 
        date_built
    ORDER BY 
        date_built
''')

avg_home_price.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

avg_price_view = spark.sql('''
    SELECT 
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    GROUP BY 
        view
    HAVING 
        avg_price >= 350000
    ORDER BY 
        view DESC
''')

avg_price_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 0.9729480743408203 seconds ---


In [8]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable('home_sales')
 
# Validate that table is cached.
spark.catalog.isCached('home_sales')

True

In [9]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

avg_price_view = spark.sql('''
    SELECT 
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    GROUP BY 
        view
    HAVING 
        avg_price >= 350000
    ORDER BY 
        view DESC
''')

avg_price_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 3.340886116027832 seconds ---


In [11]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

home_df.write.partitionBy("date_built").parquet("partitioned_home_sales")


In [12]:
# 11. Read the formatted parquet data.
read_parquet_home_sales = spark.read.parquet('partitioned_home_sales')

In [13]:
# 12. Create a temporary table for the parquet data.
read_parquet_home_sales.createOrReplaceTempView('parquet_home_sales')

In [14]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

avg_price_view = spark.sql('''
    SELECT 
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        parquet_home_sales
    GROUP BY 
        view
    HAVING 
        avg_price >= 350000
    ORDER BY 
        view DESC
''')

avg_price_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 1.3166210651397705 seconds ---


In [15]:
# 14. Uncache the home_sales temporary table.

spark.catalog.uncacheTable('home_sales')



In [16]:
# 15. Check if the home_sales is no longer cached

if spark.catalog.isCached('home_sales'):
    print("The home_sales table is still cached.")
else:
    print("The home_sales table is no longer cached.")

spark.stop()

The home_sales table is no longer cached.
